# Import thư viện

In [293]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import traceback

In [294]:
# Mở trình duyệt Google Chrome ở trang thái Tab ẩn danh
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument("--start-maximized")
s = Service('C:\chromedriver')
driver = webdriver.Chrome(service=s, options=options)
driver.implicitly_wait(0)

In [295]:
# List tất cả url movie
list_all_url_movie = pd.read_csv('E:/Crawl_Web/Rotten Tomatoes/Crawl URL/URL_movie.csv').squeeze('columns').tolist()

# Thông tin phim

In [296]:
# Mở url
# url = 'https://www.rottentomatoes.com/m/operation_fortune_ruse_de_guerre'
# url = 'https://www.rottentomatoes.com/m/bunker_2022'
url = 'https://www.rottentomatoes.com/m/beg1994'
# url = 'https://www.rottentomatoes.com/m/antman'
driver.get(url)

In [297]:
# Đợi cho đến khi url load xong
element = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[id="main_container"]')))
html_of_interest = driver.execute_script('return arguments[0].innerHTML',element)
soup = BeautifulSoup(html_of_interest, 'lxml')

In [298]:
# Tiêu đề phim
title = soup.select('h1[class="scoreboard__title"]')[0].text
title

'Beg!'

In [299]:
# Điểm đánh giá của nhà phê bình
tomatometer = soup.select('score-board')[0].attrs['tomatometerscore']
tomatometer

''

In [300]:
# Điểm đánh giá của khán giả
audience_score = soup.select('score-board')[0].attrs['audiencescore']
audience_score

''

In [301]:
# Số lượt đánh giá của nhà phê bình
num_tomatometer = soup.select('a[slot="critics-count"]')[0].text.strip()
num_tomatometer

'0 Reviews'

In [302]:
# Số lượt đánh giá của khán giả
num_audience_score = soup.select('a[slot="audience-count"]')[0].text.strip()
num_audience_score

'50+ Ratings'

In [303]:
# The Tomatometer score
tomatometer_state = soup.select('score-board')[0].attrs['tomatometerstate']
tomatometer_state

''

In [304]:
# The Audience Score
audience_state = soup.select('score-board')[0].attrs['audiencestate']
audience_state

''

In [305]:
# Thông tin phim
list_name_info = soup.select('div[class="meta-label subtle"]')
list_info = soup.select('div[data-qa="movie-info-item-value"]')

for num_info in range(len(list_name_info)):
    name_info = list_name_info[num_info].text.replace('  ', '').replace('\n', '').replace(':', '')
    info = list_info[num_info].text.replace('  ', '').replace('\n', '').replace(':', '')
    print(name_info + info)

GenreHorror
Original LanguageEnglish
DirectorRobert Golden
WriterPeta Lily, David Glass, Robert Golden
Release Date (Streaming)Oct 4, 2005
Runtime1h 48m


In [306]:
# Tóm tắt nội dụng phim
synopsis = soup.select('div[id="movieSynopsis"]')[0].text.strip()
synopsis

'Murder and mayhem unfold as two rivals jockey for position at a decrepit British hospital.'

In [307]:
# Diễn viên
list_raw_cast = soup.select('span[class="characters subtle smaller"]')
list_cast = []

for c in list_raw_cast:
    list_cast.append(c.attrs['title'])

cast = ', '.join(list_cast)
cast

'Peta Lily, Philip Pellew, Julian Bleach, Olegar Fedoro, Chris Banks, Jeremy Wilkin, Robert Golden, Peta Lily, David Glass, Robert Golden'

# Reviews

### Critic and Top critic reviews

In [308]:
# Chuyển đến url critic reviews
driver.get(url + '/reviews')

element_critic_review = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[class="layout reviews-page-container"]')))
html_of_interest = driver.execute_script('return arguments[0].innerHTML',element_critic_review)
soup_critic_review = BeautifulSoup(html_of_interest, 'lxml')

In [309]:
# Check có review của nhà phê bình không ?
check_critic_review = soup_critic_review.select('div[class="review_table"]')[0].text.strip()

if check_critic_review != '':
    list_critic_review = []
    list_top_critic_review = []
    
    # Số lượng review trong 1 trang
    num_rv = len(soup_critic_review.select('div[class="review-row"]'))

    for num in range(num_rv):
        rv = soup_critic_review.select('div[class="review-row"]')[num].find_all('p', {'class': 'review-text'})[0].text
        list_critic_review.append(rv)

        # Check top critic reviews
        check_top_critic = soup_critic_review.select('div[class="review-row"]')[num].find_all('rt-icon-top-critic')

        if check_top_critic != []:
            list_top_critic_review.append(rv)
    
    while True:
        sleep(1)
        try:
            next_button = driver.find_element(By.CSS_SELECTOR, 'rt-button[class="js-prev-next-paging-next"]')
            driver.execute_script("arguments[0].click();", next_button)
            
            element_critic_review = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[class="review_table"]')))
            html_of_interest = driver.execute_script('return arguments[0].innerHTML',element_critic_review)
            soup_critic_review = BeautifulSoup(html_of_interest, 'lxml')

            num_rv = len(soup_critic_review.select('div[class="review-row"]'))

            for num in range(num_rv):
                rv = soup_critic_review.select('div[class="review-row"]')[num].find_all('p', {'class': 'review-text'})[0].text.strip()
                list_critic_review.append(rv)

                check_top_critic = soup_critic_review.select('div[class="review-row"]')[num].find_all('rt-icon-top-critic')

                if check_top_critic != []:
                    list_top_critic_review.append(rv)
            
        except Exception:
            # traceback.print_exc()
            break

    critic_review = '<>'.join(list_critic_review)
    if list_top_critic_review != []:
        top_critic_review = '<>'.join(list_top_critic_review)
    else:
        top_critic_review = 'No top critic reviews'
    
    print(critic_review)
    print(top_critic_review)
else:
    critic_review = 'No critic reviews'
    print(critic_review)

No critic reviews


### Audience and Verified audience reviews

In [310]:
# Chuyển đến url review của khán giả
driver.get(url + '/reviews?type=user')

element_audience_review = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-qa="reviews-container"]')))
html_of_interest = driver.execute_script('return arguments[0].innerHTML',element_audience_review)
soup_audience_review = BeautifulSoup(html_of_interest, 'lxml')

In [311]:
# Check có review của khán giả không ?
check_audience_review = soup_audience_review.select('div[id="movieUserReviewsContent"]')

if len(check_audience_review) != 0:
    list_audience_review = []
    
    for rv in soup_audience_review.select('p[data-qa="review-text"]'):
        list_audience_review.append(rv.text)
    
    while True:
        sleep(1)
        try:
            next_button = driver.find_element(By.CSS_SELECTOR, 'rt-button[class="js-prev-next-paging-next"]')
            driver.execute_script("arguments[0].click();", next_button)
            
            element_audience_review = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[id="movieUserReviewsContent"]')))
            html_of_interest = driver.execute_script('return arguments[0].innerHTML', element_audience_review)
            soup_audience_review = BeautifulSoup(html_of_interest, 'lxml')

            for rv in soup_audience_review.select('p[data-qa="review-text"]'):
                list_audience_review.append(rv.text)
            
        except:
            break
    audience_review = '<>'.join(list_audience_review)
    print(audience_review)
else:
    audience_review = 'Không có review của khán giả'
    print(audience_review)

I'm not really sure what to say about this.  It was like watching someone else's nightmare, starring a woman who looks like Helen Hunt and a bunch of really nutty 80s-British-sitcom music and really dramatic 90s-soap-opera music.
Also dogs.  Also a lot of gross eating.  Also weird sex scenes.  Also cartoon-inspired comedy scenes.
It really makes almost zero sense.  Especially the whole hospital/jazz bar/bum park/wharf thing.  And the remote-control cop thing.  And then the live dogs sewn into live humans thing.  And the "dogfish" thing.  And the monster-in-the-rose-garden thing.  Oh man.  There was nothing that didn't boggle my mind.  
Except the reveal of the murderer.  I knew who the murderer was pretty much the whole time.  I'm not sure if that was the point or not.  Was there a point?
WTF?
